In [46]:
import pandas as pd

df = pd.read_csv("./extracted_data/Cleaned_Standard_stats.csv", usecols=["Key", "NonPenaltyGoalsPer90", "YelloCards", "ProgressiveCarries", "ProgressivePasses", "Min", "Goals", "Assists"])
df1 = pd.read_csv("./extracted_data/Cleaned_Passing.csv", usecols=["Key", "Cmp%"])
df2 = pd.read_csv("./extracted_data/Cleaned_Goal_and_shot.csv", usecols=["Key", "GCA90", "SCA90"])
df3 = pd.read_csv("./extracted_Data/Cleaned_Defensive.csv", usecols=["Key", "Total_Tackles", "Tackles_Won"])

# Ta bort duplicerade 'Key' värden
df = df.drop_duplicates(subset="Key")
df1 = df1.drop_duplicates(subset="Key")
df2 = df2.drop_duplicates(subset="Key")
df3 = df3.drop_duplicates(subset="Key")

df["Season"] = df["Key"].str[:9]
df1["Season"] = df1["Key"].str[:9]
df2["Season"] = df2["Key"].str[:9]
df3["Season"] = df3["Key"].str[:9]

df_merged1 = pd.merge(df, df1, on=["Key", "Season"], how="left")
df_merged2 = pd.merge(df_merged1, df2, on=["Key", "Season"], how="left")
df_merged3 = pd.merge(df_merged2, df3, on=["Key", "Season"], how="left")

df_merged3["NonPenaltyGoalsPer90"] = pd.to_numeric(df_merged3["NonPenaltyGoalsPer90"], errors='coerce')
df_merged3["YelloCards"] = pd.to_numeric(df_merged3["YelloCards"], errors="coerce")
df_merged3["ProgressiveCarries"] = pd.to_numeric(df_merged3["ProgressiveCarries"], errors="coerce")
df_merged3["ProgressivePasses"] = pd.to_numeric(df_merged3["ProgressivePasses"], errors="coerce")
df_merged3["Min"] = pd.to_numeric(df_merged3["Min"], errors='coerce')
df_merged3["Goals"] = pd.to_numeric(df_merged3["Goals"], errors='coerce')
df_merged3["Assists"] = pd.to_numeric(df_merged3["Assists"], errors='coerce')
df_merged3["Cmp%"] = pd.to_numeric(df_merged3["Cmp%"], errors="coerce")
df_merged3["GCA90"] = pd.to_numeric(df_merged3["SCA90"], errors="coerce")
df_merged3["SCA90"] = pd.to_numeric(df_merged3["SCA90"], errors="coerce")
df_merged3["Total_Tackles"] = pd.to_numeric(df_merged3["Total_Tackles"], errors="coerce")
df_merged3["Tackles_Won"] = pd.to_numeric(df_merged3["Tackles_Won"], errors="coerce")

df_merged3["Tackle%"] = df_merged3["Tackles_Won"] / df_merged3["Total_Tackles"]
df_merged3["Goals_90"] = df_merged3["Goals"] / 90
df_merged3["Assists_90"] = df_merged3["Assists"] / 90
df_merged3["Player"] = df_merged3["Key"].str[10:]

df_merged3.drop("Goals", axis=1)
df_merged3.drop("Assists", axis=1)
df_merged3.drop("Total_Tackles", axis=1)
df_merged3.drop("Tackles_Won", axis=1)

# Standardisera varje relevant kolumn (Z-score)
df_merged3["Goals_z"] = (df_merged3["Goals"] - df_merged3["Goals"].mean()) / df_merged3["Goals"].std()
df_merged3["Assists_z"] = (df_merged3["Assists"] - df_merged3["Assists"].mean()) / df_merged3["Assists"].std()
df_merged3["YelloCards_z"] = (df_merged3["YelloCards"] - df_merged3["YelloCards"].mean()) / df_merged3["YelloCards"].std()
df_merged3["ProgressiveCarries_z"] = (df_merged3["ProgressiveCarries"] - df_merged3["ProgressiveCarries"].mean()) / df_merged3["ProgressiveCarries"].std()
df_merged3["ProgressivePasses_z"] = (df_merged3["ProgressivePasses"] - df_merged3["ProgressivePasses"].mean()) / df_merged3["ProgressivePasses"].std()
df_merged3["NonPenaltyGoalsPer90_z"] = (df_merged3["NonPenaltyGoalsPer90"] - df_merged3["NonPenaltyGoalsPer90"].mean()) / df_merged3["NonPenaltyGoalsPer90"].std()
df_merged3["Cmp%_z"] = (df_merged3["Cmp%"] - df_merged3["Cmp%"].mean()) / df_merged3["Cmp%"].std()
df_merged3["SCA90_z"] = (df_merged3["SCA90"] - df_merged3["SCA90"].mean()) / df_merged3["SCA90"].std()
df_merged3["GCA90_z"] = (df_merged3["GCA90"] - df_merged3["GCA90"].mean()) / df_merged3["GCA90"].std()
df_merged3["Tackle%_z"] = (df_merged3["Tackle%"] - df_merged3["Tackle%"].mean()) / df_merged3["Tackle%"].std()
df_merged3["Goals_90_z"] = (df_merged3["Goals_90"] - df_merged3["Goals_90"].mean()) / df_merged3["Goals_90"].std()
df_merged3["Assists_90_z"] = (df_merged3["Assists_90"] - df_merged3["Assists_90"].mean()) / df_merged3["Assists_90"].std()

# Beräkna den sammansatta ratingen med de standardiserade värdena
df_merged3["PlayerRating"] = (
    0.1 * df_merged3["ProgressiveCarries_z"] + 
    0.1 * df_merged3["ProgressivePasses_z"] + 
    0.1 * df_merged3["NonPenaltyGoalsPer90_z"] + 
    0.15 * df_merged3["Cmp%_z"] + 
    0.05 * df_merged3["SCA90_z"] + 
    0.05 * df_merged3["GCA90_z"] + 
    0.10 * df_merged3["Tackle%_z"] + 
    0.2 * df_merged3["Goals_90_z"] + 
    0.15 * df_merged3["Assists_90_z"] +
    (-0.05) * df_merged3["YelloCards_z"]  
)

min_rating = df_merged3["PlayerRating"].min()
max_rating = df_merged3["PlayerRating"].max()

df_merged3["PlayerRating_1_10"] = 0.9764 + 9 * (df_merged3["PlayerRating"] - min_rating) / (max_rating - min_rating) # Denna uträkning gör att det hamnar på en 1-10 skala

print(df_merged3[["Player", "PlayerRating_1_10", "Season"]].sort_values(by="PlayerRating_1_10", ascending=False).dropna().head(10))



             Player  PlayerRating_1_10     Season
5178    lionelmessi           9.976400  2018-2019
9385    lionelmessi           9.825482  2019-2020
1132    lionelmessi           9.739025  2017-2018
24805    dusantadic           9.511734  2022-2023
18805  kylianmbappe           9.430187  2021-2022
7039     dusantadic           9.176295  2018-2019
23392  kylianmbappe           9.097750  2022-2023
13656   lionelmessi           9.058305  2020-2021
20235    dusantadic           8.926330  2021-2022
18082  karimbenzema           8.844964  2021-2022
